# IAU - Inteligentná analýza údajov (2024/2025)

#### Autori: Jan Lenhart (50 %), Marek Čederle (50 %)
##### Cvičenie: Pondelok 15:00, Cvičiaci: Ing. Oleksandr Lytvyn

## Fáza 1 - Prieskumná analýza

Na začiatok si importujeme knižnice a načítame dáta do dátových rámcov (dataframov).

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from copy import copy
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp

In [ ]:
pd.set_option("display.width", 2**20)
pd.set_option("display.max_columns", 32)

In [ ]:
df_connections = pd.read_csv('112/connections.csv', delimiter=',')
# keep_default_na=False  bolo treba aby nacitalo dobry format pre 'NA' string v devices.csv
df_devices     = pd.read_csv('112/devices.csv', delimiter=',', keep_default_na=False)
df_processes   = pd.read_csv('112/processes.csv', delimiter=',')
df_profiles    = pd.read_csv("112/profiles.csv", delimiter=',')

### 1.1 Základný opis dát spolu s ich charakteristikami

#### 1.1.A Analýza štruktúry dát

##### Štruktúra dát - počty riadkov a stĺpcov, dátové typy

In [ ]:
df_connections.info()

Data frame `df_connections`
- Obsahuje 15126 riadkov (záznamov) a 13 stĺpcov.
- Všetky stĺpce okrem `ts` a `imei` sú typu `float64` čiže nejaké reálne čísla.
- Stĺpec `ts` je typu `object` čiže nejaký zložitejší dátový typ. (V podstate `string`)
- Stĺpec `imei` je typu `int64` čiže celé číslo.

In [ ]:
df_devices.info()

Data frame `df_devices`
- Obsahuje 2947 riadkov (záznamov) a 6 stĺpcov
- Stĺpce `latitude` a `longitude` sú typu `float64` čiže nejaké reálne čísla.
- Stĺpce `store_name`, `code` a `location` sú typu `object` čiže nejaký zložitejší dátový typ. (V podstate `string`)
- Stĺpec `imei` je typu `int64` čiže celé číslo.

In [ ]:
df_processes.info()

Data frame `df_processes`
- Obsahuje 15126 riadkov (záznamov) a 23 stĺpcov

- Všetky stĺpce okrem `ts` a `imei` sú typu `float64` čiže nejaké reálne čísla.
- Stĺpec `ts` je typu `object` čiže nejaký zložitejší dátový typ. (V podstate `string`)
- Stĺpec `imei` je typu `int64` čiže celé číslo.

In [ ]:
df_profiles.info()

Data frame `df_profiles`
- Obsahuje 2553 riadkov (záznamov) a 12 stĺpcov

- Všetky stĺpce okrem `user_id` a `imei` sú typu `object` čiže nejaký zložitejší dátový typ. (V podstate `string`)
- Stĺpece `imei` a `user_id` sú typu `int64` čiže celé číslo.

IMEI je unikátny identifikátor mobilného zariadenia, ktorý sa nachádza vo všetkých rámcoch, takže ho bude možné použiť ako primárny kľúč na spájanie tabuliek, ak to bude neskôr potrebné.

##### Ukážka dát z daných data framov ako tabuľka záznamov

**Connections**

In [ ]:
df_connections.head()

**Devices**

In [ ]:
df_devices.head()

**Processes**

In [ ]:
df_processes.head()

**Profiles**

In [ ]:
df_profiles.head()

#### 1.1.B Analýza jednotlivých atribútov

##### Deskriptívne štatistiky (pre dataframy) a distribúcie pre vybrané atribúty

**Connections**

In [ ]:
df_connections.describe()

In [ ]:
df_connections['mwra'].value_counts()

Môžeme si všimnúť že stĺpec `mwra` má hodnotu 0 alebo 1 s tým, že hodnôt 1 je viacej ako hodnôt 0.

Taktiež vidíme že všetky stĺpce okrem `imei` a `mwra` majú hodnoty v rozsahu od 0 do 100, s tým že niektoré majú interval viacej ohraničený.

In [ ]:
sns.histplot(df_connections['c.katana'], kde=True, color='blue')

In [ ]:
stats.probplot(df_connections["c.katana"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_connections['c.katana'])

In [ ]:
sns.histplot(df_connections['c.android.gm'], kde=True, color='red')

In [ ]:
stats.probplot(df_connections["c.android.gm"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_connections['c.android.gm'])

In [ ]:
sns.histplot(df_connections['c.android.chrome'], kde=True, color='yellow')

In [ ]:
stats.probplot(df_connections["c.android.chrome"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_connections['c.android.chrome'])

In [ ]:
sns.histplot(df_connections['c.dogalize'], kde=True, color='green')

In [ ]:
stats.probplot(df_connections["c.dogalize"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_connections['c.dogalize'])

Vybrané stĺpce z dátoveho rámca `df_connections` boli tie, ktoré mali významnú koreláciu(informácia získaná v 1.1.D). Distribúcia na histograme stĺpca `c.katana` a stĺpca `c.android.chrome` sa naviac podobá normálnej distribúcii v porovnaní s ostatnými distribúciami vybraných stĺpcov z `df_connections`. Síce bolo vidieť, že okraje (tails) distribúcie nevyzerajú na okraje normálnej distribúcie. Preto sme si urobili `qqplot`, na ktorom bolo vidieť podstatný rozdiel normálovou distribúciou. Môžeme jedine povedať, že sa to podobá normálovej distribúcie a potvrdiť to štatistickým testom.

Taktiež vidíme modus vybraných stĺpcov z dátoveho rámca `df_connections` čo sú vlastne najviac vyskztujúce sa hodnoty.

In [ ]:
def print_normality_test(df):
    if df.count() > 5000:
        stat, p_value = stats.kstest(df, 'norm')
        print(f"kstest for count={df.count()}: [stat: {stat}, p: {p_value}, norm: {p_value > 0.05}]")
    else:
        stat, p_value = stats.shapiro(df)
        print(f"stat: for count={df.count()}: [stat: {stat}, p: {p_value}, norm: {p_value > 0.05}]")

In [ ]:
print_normality_test(df_connections["c.katana"])
print_normality_test(df_connections["c.android.gm"])
print_normality_test(df_connections["c.android.chrome"])
print_normality_test(df_connections["c.dogalize"])

Počet záznamov je väčší ako 5000, preto bol použitý Kolmogorov-Smirnov (`kstest`) namiesto Shapiro-Wilk z ktorého vyplíva, že dáta vo vybraných stĺpcoch nespĺňajú normálovú distribúciu pretože p-hodnota je menšia ako 0.05.

**Devices**

In [ ]:
df_devices.describe()

- Máme štatistiku iba z atribútov, ktoré majú numerický dátový typ.
- Atribúty `latitude` a `longitude` sú zemepisné súradnice a ich deskriptívna štatistika nemá veľký význam pri týchto dátach pretože sú to zemepisné súradnice a nie numerické hodnoty s ktorými by sme mohli ďalej pracovať. Informácia o tom, aký je napr. mean pre zemepisnú šírku nám nedáva žiadny význam.
- Atribút `imei` je unikátny identifikátor mobilného zariadenia a jeho deskriptívna štatistika nemá veľký význam.

**Processes**

In [ ]:

df_processes.describe()

In [ ]:
df_processes['mwra'].value_counts()

Môžeme si všimnúť že stĺpec `mwra` má hodnotu 0 alebo 1 s tým, že hodnôt 1 je viacej ako hodnôt 0.

Taktiež vidíme že všetky stĺpce okrem `imei` a `mwra` majú hodnoty v rozsahu od 0 do 100, s tým že niektoré majú interval viacej ohraničený.

In [ ]:
sns.histplot(df_processes['p.android.gm'], kde=True, color='blue')

In [ ]:
stats.probplot(df_processes["p.android.gm"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_processes['p.android.gm'])

In [ ]:
sns.histplot(df_processes['p.system'], kde=True, color='red')

In [ ]:
stats.probplot(df_processes["p.system"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_processes['p.system'])

In [ ]:
sns.histplot(df_processes['p.android.chrome'], kde=True, color='yellow')

In [ ]:
stats.probplot(df_processes["p.android.chrome"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_processes['p.android.chrome'])

In [ ]:
sns.histplot(df_processes['p.browser.provider'], kde=True, color='green')

In [ ]:
stats.probplot(df_processes["p.browser.provider"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_processes['p.browser.provider'])

In [ ]:
sns.histplot(df_processes['p.android.documentsui'], kde=True, color='magenta')

In [ ]:
stats.probplot(df_processes["p.android.documentsui"], dist="norm", plot=plt)

In [ ]:
stats.mode(df_processes['p.android.documentsui'])


In [ ]:
sns.histplot(df_processes['p.android.packageinstaller'], kde=True, color='grey')

In [ ]:
stats.probplot(df_processes["p.android.packageinstaller"], dist="norm", plot=plt)


In [ ]:
stats.mode(df_processes['p.android.packageinstaller'])


Podobne ako pre `df_connections` rovnako v `df_processes` sme vybrali stĺpce, ktoré mali významnú koreláciu (zistené v 1.1.D). Distribúcia na histograme stĺpca `p.android.documentsui` sa naviac podobá normálnej distribúcii v porovnaní s ostatnými distribúciami vybraných stĺpcov z `df_processes`. Niektoré distribúcie bolo hneď možné identifikovať že nejde o normálne distribúcie, každopádne sme pre všetky urobili `qqplot`, na ktorom bolo vidieť podstatný rozdiel normálovou distribúciou.

Taktiež vidíme modus vybraných stĺpcov z dátoveho rámca `df_processes` čo sú vlastne najviac vyskytujúce sa hodnoty.

In [ ]:
print_normality_test(df_processes["p.android.gm"])
print_normality_test(df_processes["p.system"])
print_normality_test(df_processes["p.android.chrome"])
print_normality_test(df_processes["p.browser.provider"])
print_normality_test(df_processes["p.android.documentsui"])
print_normality_test(df_processes["p.android.packageinstaller"])


Počet záznamov je väčší ako 5000, preto bol použitý Kolmogorov-Smirnov (`kstest`) namiesto Shapiro-Wilk z ktorého vyplíva, že dáta vo vybraných stĺpcoch nespĺňajú normálovú distribúciu pretože p-hodnota je menšia ako 0.05.

**Profiles**

In [ ]:
df_profiles.describe()

- Máme štatistiku iba z atribútov, ktoré majú numerický dátový typ, avšak keďže ide o unikátne identifikátory, tak tieto štatistiky nemajú žiadny význam.

#### 1.1.C Párová analýza dát

In [ ]:
# opens image generated by commented out sns.pairplot calls
# it is done this way because the graphs are very big in size and take very long time to generate

In [ ]:
#sns.pairplot(df_connections)
Image.open('plots/pairplot_connections.png')

In [ ]:
# sns.pairplot(df_devices)
Image.open('plots/pairplot_devices.png')

In [ ]:
#sns.pairplot(df_processes)
Image.open('plots/pairplot_processes.png')

In [ ]:
# sns.pairplot(df_profiles)
Image.open('plots/pairplot_profiles.png')

V `df_connections` nie je okom viditeľná žiadna korelácia a distribúcie stĺpcových hodnôt nadobúdajú rôzne typy ako sú uniformná, normálna (niektoré sa trochu podobajú a iné zasa sú 'skewed').
V `df_devices` nemá veľa numerických stĺpcov a nedáva nič hodnotné. Jedine sa dá vyčítať z akých častí sveta je aké množstvo záznamov.
V `df_processes` je veľmi podobné ako v `df_connections`, až na to že je viditeľná nelineárna korelácia medzi `p.browser.provider` a `p.system`. Taktiež distribúcia `p.browser.provider` je veľmi 'skewed'.
V `df_profiles` podobne ako `df_devices` nedáva nič zmysluplné.

#### 1.1.D Párová analýza dát medzi predikovanou premennou

In [ ]:
target_predictor = 'mwra'
strong_predictors = pd.DataFrame()
moderate_predictors = pd.DataFrame()
weak_predictors = pd.DataFrame()

In [ ]:
def numerical_prediction(df):
    global strong_predictors, moderate_predictors, weak_predictors
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
    corr_matrix = df[numerical_columns].corr()
    corrs = corr_matrix[[target_predictor]].sort_values(by=target_predictor, ascending=False)
    corrs = corrs.drop('mwra')
    sns.heatmap(corrs, annot=True, cmap='coolwarm')
    plt.title(f'Correlation of {target_predictor} with numerical predictors')
    plt.show()
    strong_predictors = pd.concat([strong_predictors, pd.DataFrame(np.where(abs(corrs) > 0.7, corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])
    moderate_predictors = pd.concat([moderate_predictors, pd.DataFrame(np.where((abs(corrs) > 0.5) & (abs(corrs) <= 0.7), corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])
    weak_predictors = pd.concat([weak_predictors, pd.DataFrame(np.where((abs(corrs) > 0.25) & (abs(corrs) <= 0.5), corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])

Funkcia `numerical_prediction` získa prediktory pre dátový rámec, ktorý dostane ako parameter. Získané prediktory akumuluje do globálnych premenných `strong_predictors`, `moderate_predictors` a `weak_predictors`. To sú kategorizované premenné, ktoré v páre so stĺpcom `mwra` majú koreláciu viac ako `0.7` pre `strong`, viac ako `0.5` a menej ako `0.7` pre `moderate` a viac ako `0.25` (bolo by lepšie, keby toto číslo bolo 0.3, ale chceli sme zvýšiť počet prediktorov a ak budú dávať zlé výsledky, môžeme ich kedykoľvek vyhodiť) a menej ako `0.5` pre `weak`. Taktiež, pri volaní zobrazí heatmapu vypočítaných korelácií, aby poskytla vizuálny prehľad o vzťahoch medzi premennými.

In [ ]:
numerical_prediction(df_connections)

Tento graf nám hovorí o tom, ktoré premenné z dátového rámca `df_connections` korelujú s našou predikovanou premennou `mwra`.
Môžeme vidieť že stĺpce `c.dogalize`, `c.android.chrome` a `c.katana` majú najvyššiu koreláciu s predikovanou premennou `mwra`.

In [ ]:
numerical_prediction(df_processes)

Tento graf nám hovorí o tom, ktoré premenné z dátového rámca `df_processes` korelujú s našou predikovanou premennou `mwra`.
Môžeme vidieť že stĺpce `p.android.gm`, `p.system`, `p.android.chrome`, `p.android.documentsui` a `p.android.packageinstaller` majú najvyššiu koreláciu s predikovanou premennou `mwra`.

In [ ]:
print(strong_predictors)

In [ ]:
print(moderate_predictors)

In [ ]:
print(weak_predictors)

Tu môžeme vidieť že nemáme žiadne silné predikátory (korelácia `> 0.7`).
Máme však stredne silné a slabšie predikátory.

#### 1.1.E Zamyslenie k riešeniu projektu

Pomocou párovej analýzi sme zistili nejakú koreláciu resp. závisloť premenných. Konkrétne mali najsilnejšiu koreláciu s predikovanou premennou `mwra` tieto stĺpce:
- c.dogalize
- p.android.gm
- p.android.packageinstaller

Kedže sa nachádzajú v iných dátových rámcoch, bude potrebné ich spojiť pomocou `imei`.

Podľa toho, ako dáta vyzerajú a čo sme zistili, tak si myslíme že ide o nejaké využitie CPU s tým že CPU môže mať viacero jadier a preto hodnoty premenných nedávajú súčet 100 ale viacej čo by znamenalo že by mohlo ísť o viacej jadrové CPU (napr. 8), ale toto je iba naša teória.

Je vysoko pravdepodobné že bude treba spájať dané dáta pretože aj `devices.csv` aj `processes.csv` majú stĺpec `mwra` a budeme muset zistit koreláciu naprieč týmito dátami.

### 1.2 Identifikácia problémov, integrácia a čistenie dát

#### 1.2.A Identifikácia a prvotné riešenie problémov v dátach

##### Namapovanie premenných typu `string` na numerické hodnoty

In [ ]:
df_devices.info()

In [ ]:
connections_string_columns = set(copy(df_connections.select_dtypes(include=['object']).columns))
devices_string_columns = set(copy(df_devices.select_dtypes(include=['object']).columns))
processes_string_columns = set(copy(df_processes.select_dtypes(include=['object']).columns))
profiles_string_columns = set(copy(df_profiles.select_dtypes(include=['object']).columns))
string_columns = list(connections_string_columns | devices_string_columns | processes_string_columns | profiles_string_columns)
print(string_columns)

##### Vymazanie duplikátov v dátach

In [ ]:
def remove_duplicates(df):
    pre_count = df.duplicated().count()
    df = df.drop_duplicates()
    post_count = df.duplicated().count()
    print(f"removed {pre_count - post_count} duplicates")
    return df

In [ ]:
df_connections = remove_duplicates(df_connections)

In [ ]:
df_devices = remove_duplicates(df_devices)

In [ ]:
df_processes = remove_duplicates(df_processes)

In [ ]:
df_profiles = remove_duplicates(df_profiles)

#### 1.2.B Chýbajúce hodnoty (missing values)

In [ ]:
def printNonesAndNA(label, df):
    nulls = df.isnull().sum()
    if (nulls.sum()):
        df_temp = pd.DataFrame(np.where(nulls > 0, nulls, np.nan), index=nulls.index, columns=['null_count']).dropna()
        df_temp['percentage'] = df_temp.apply(lambda x: x / df.shape[0] * 100)
        print(label)
        print(df_temp)
        print()
        return df_temp

In [ ]:
printNonesAndNA("df_connections", df_connections)
printNonesAndNA("df_devices", df_devices)
printNonesAndNA("df_processes", df_processes)
profiles_null_columns = printNonesAndNA("df_profiles", df_profiles)

Z tejto funckie sme zistili, že nám chýbajú hodnoty iba z dataframu `df_profiles`, ktoré sú výhradne typu `string` a ich počet vzhľadom ku celkovému počtu záznamov je veľmi veľký, takže dané stĺpce môžeme odstrániť pretože by pre nás nemali žiadnu hodnotu. Kedže sú to nie numerické dáta, tak by sme ich nemohli nijako nahradiť.

In [ ]:
df_profiles.drop(columns=profiles_null_columns.index, axis=1)

#### 1.2.C Vychýlené hodnoty (outlier detection)

Na ukážku si vykreslíme boxploty pre vybrané atribúty a zistíme, či sa v nich nachádzajú nejaké vychýlené hodnoty. Naše atribúty sú premenné, ktoré najviac korelovali s predikovanou premennou `mwra` a teda sú to:
- c.dogalize
- p.android.gm
- p.android.packageinstaller

In [ ]:
sns.boxplot(x='mwra', y='c.dogalize', data=df_connections)

In [ ]:
sns.boxplot(x='mwra', y='p.android.gm', data=df_processes)

In [ ]:
sns.boxplot(x='mwra', y='p.android.packageinstaller', data=df_processes)

Následne identifikujeme outlierov pomocou IQR metódy a overíme že ich skutočne máme.

In [ ]:
# zdroj z cvicenia
def identify_outliers(data):
    lower = data.quantile(0.25) - 1.5 * stats.iqr(data)
    upper = data.quantile(0.75) + 1.5 * stats.iqr(data)
    return data[(data > upper) | (data < lower)]

In [ ]:
def identify_outliers_for_all_columns(df):
    val = {}
    columns = df.select_dtypes(exclude=['object']).columns
    for column in columns:
        outlier_count = identify_outliers(df[column]).count()
        if outlier_count > 0:
            val[column] = outlier_count
    return val

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_devices)

In [ ]:
identify_outliers_for_all_columns(df_processes)

In [ ]:
identify_outliers_for_all_columns(df_profiles)

Zistili sme že ich máme relatívne málo vzhľadom na počet záznamov, takže ich môžeme odstrániť.

In [ ]:
def remove_outliers_for_all_columns(df):
    df_temp = df.copy()
    indices = pd.DataFrame({})
    accumulated_indices = set()
    for column in df_temp.select_dtypes(exclude=['object']).columns:
        accumulated_indices |= set(identify_outliers(df_temp[column]).index)
    df_temp = df_temp.drop(accumulated_indices)
    return df_temp

In [ ]:
df_connections = remove_outliers_for_all_columns(df_connections)
# df_devices = remove_outliers_for_all_columns(df_devices)
df_processes = remove_outliers_for_all_columns(df_processes)
# df_profiles = remove_outliers_for_all_columns(df_profiles)

Ostránime iba outlierov z dátového rámca `df_processes` a `df_connections` pretože, v dátovom rámci `df_devices` a `df_profiles` nemáme žiadne numerické hodnoty, s ktorými je vhodné pracovať. (Zemepisné súradnice a unikátne identifikátory)
Po odstránení outlierov si možeme znova identifikovať či náhodou nejakých ešte nemáme.

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_processes)

Zistili sme že napriek úvodnému odstráneniu mám stále outlierov. Deje sa to z toho dôvodu, že sa vlastne distibúcia posunie a preto sa nám objavili nový outliery. Keby takto pokračujeme a odstraňujeme stále dáta, tak by sme prišli o veľký počet záznamov a preto sme sa rozhodli ich nahradiť hraničnými hodnotami (5% a 95%).

In [ ]:
sns.boxplot(x='mwra', y='p.browser.provider', data=df_processes)

In [ ]:
sns.histplot(df_processes['p.browser.provider'], kde=True, color='blue')

In [ ]:
def replace_outliers_with_percentiles(df):
    df_temp = df.copy()
    for column in df_temp.select_dtypes(exclude=['object']).columns:
        lower_bound = df_temp[column].quantile(0.05)
        upper_bound = df_temp[column].quantile(0.95)
        lower_outliers = df_temp[column] < lower_bound
        upper_outliers = df_temp[column] > upper_bound
        df_temp.loc[lower_outliers, column] = lower_bound
        df_temp.loc[upper_outliers, column] = upper_bound
    return df_temp

In [ ]:
df_connections = replace_outliers_with_percentiles(df_connections)
df_devices = replace_outliers_with_percentiles(df_devices)
df_processes = replace_outliers_with_percentiles(df_processes)
df_profiles = replace_outliers_with_percentiles(df_profiles)

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_devices)

In [ ]:
identify_outliers_for_all_columns(df_processes)

In [ ]:
identify_outliers_for_all_columns(df_profiles)

In [ ]:
# histogram
sns.histplot(df_processes['p.browser.provider'], kde=True, color='blue')

Ako vidíme z výpisu dát, tak sa nám podarilo všetkých outlierov bud odstrániť alebo nahradiť hraničnými hodnotami.
Jedniou výnimkou je `p.browser.provider` a to z dôvodu ako vyzerá jeho distribúcia. Preto sme sa rozhodli to nemeniť, pretože by sme pri iteratívnom odstraňovaní prišli o veľký počet záznamov.

### 1.3 Formulácia a štatistické overenie hypotéz o dátach

#### 1.3.A Formulácia hypotéz

##### Hypotéza č.1

Zvolíme si náš "significance level" na $\alpha = 0.05$. (95%)

Null hypothesis (nulová hypotéza):

$H_0$: Premenná `p.android.gm` má v priemere rovnakú váhu v stave malware-related-activity ako v normálnom stave.

Alternative hypothesis (alternatívna hypotéza):
$H_1$ = $H_A$: Premenná `p.android.gm` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave. (Nižšiu alebo vyššiu)

**Najskôr si musíme overiť či dáta spĺňajú normálovú distribúciu**

Takto vyzerá náš boxplot, ktorý nám ukazuje distribúciu hodnôt pre premennú `p.android.gm` v závislosti od `mwra`.

In [ ]:
sns.boxplot(x='mwra', y='p.android.gm', data=df_processes)

In [ ]:
p_android_gm_0 = df_processes.loc[df_processes.mwra == 0, 'p.android.gm']
p_android_gm_0.describe()

In [ ]:
sns.histplot(p_android_gm_0)

In [ ]:
p_android_gm_1 = df_processes.loc[df_processes.mwra == 1, 'p.android.gm']
p_android_gm_1.describe()

In [ ]:
sns.histplot(p_android_gm_1)

Z týchto distribúcií môžeme vidieť, že nevyzerajú ako normálne distribúcie, preto musíme pokračovať s overením.

In [ ]:
p_android_gm_0_outliers = identify_outliers(p_android_gm_0)
p_android_gm_1_outliers = identify_outliers(p_android_gm_1)

In [ ]:
p_android_gm_0_outliers.count()

In [ ]:
p_android_gm_1_outliers.count()

In [ ]:
p_android_gm_0 = p_android_gm_0.drop(p_android_gm_0_outliers.index)
p_android_gm_1 = p_android_gm_1.drop(p_android_gm_1_outliers.index)

In [ ]:
sns.histplot(p_android_gm_1)

Vidíme že aj po vyhodení outlierov sa nám distribúcia nezmenila a preto môžeme pokračovať s testovaním.

In [ ]:
_ = sm.ProbPlot(p_android_gm_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(p_android_gm_1, fit=True).qqplot(line='45')


Ani `QQ-plot` nám nepotvrdil normálnu distribúciu. Pretože aby bola distribúcia normálna, tak by sa body museli nachádzať na priamke definovanej ako $x=y$.

In [ ]:
p_android_gm_0.count()

In [ ]:
p_android_gm_1.count()

Musíme použiť štatistický test, ktorý nám povie či dáta majú normálnu distribúciu alebo nie. Použijeme `Kolmogorov-Smirnov` test namiesto `Shapiro-Wilk` pretože počet záznamov je väčší ako 5000.

In [ ]:
print_normality_test(p_android_gm_0)

In [ ]:
print_normality_test(p_android_gm_1)

Výsledok testu nám potvrdil, že dáta nemajú normálnu distribúciu, preto musíme použiť neparametrický test na overenie hypotézy. Pretože p hodnota je menšia ako 0.05.

Použijeme `Mann-Whitney U Test` pretože máme práve dve premenné a zároveň spĺňajú predpoklady pre tento test (vzorka musí byť aspoň 20).

In [ ]:
stats.mannwhitneyu(p_android_gm_0, p_android_gm_1)

Keďže nám vyšlo p menšie ako 0.05 tak zamietame nulovú hypotézu a prijímame alternatívnu hypotézu, že premenná `p.android.gm` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave.

##### Hypotéza č.2

Zvolíme si náš "significance level" na $\alpha = 0.05$. (95%)

Null hypothesis (nulová hypotéza):

$H_0$: Premenná `p.android.packageinstaller` má v priemere rovnakú váhu v stave malware-related-activity ako v normálnom stave.

Alternative hypothesis (alternatívna hypotéza):
$H_1$ = $H_A$: Premenná `p.android.packageinstaller` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave. (Nižšiu alebo vyššiu)

**Najskôr si musíme overiť či dáta spĺňajú normálovú distribúciu**

Takto vyzerá náš boxplot, ktorý nám ukazuje distribúciu hodnôt pre premennú `p.android.packageinstaller` v závislosti od `mwra`.

In [ ]:
sns.boxplot(x='mwra', y='p.android.packageinstaller', data=df_processes)

In [ ]:
p_android_packageinstaller_0 = df_processes.loc[df_processes.mwra == 0, 'p.android.packageinstaller']
p_android_packageinstaller_0.describe()

In [ ]:
sns.histplot(p_android_packageinstaller_0)

In [ ]:
p_android_packageinstaller_1 = df_processes.loc[df_processes.mwra == 1, 'p.android.packageinstaller']
p_android_packageinstaller_1.describe()

In [ ]:
sns.histplot(p_android_packageinstaller_1)

Z týchto distribúcií môžeme vidieť, že nevyzerajú ako normálne distribúcie, preto musíme pokračovať s overením.

In [ ]:
p_android_packageinstaller_0_outliers = identify_outliers(p_android_packageinstaller_0)
p_android_packageinstaller_1_outliers = identify_outliers(p_android_packageinstaller_1)

In [ ]:
p_android_packageinstaller_0_outliers.count()

In [ ]:
p_android_packageinstaller_1_outliers.count()

In [ ]:
p_android_packageinstaller_0 = p_android_packageinstaller_0.drop(p_android_packageinstaller_0_outliers.index)
p_android_packageinstaller_1 = p_android_packageinstaller_1.drop(p_android_packageinstaller_1_outliers.index)

In [ ]:
sns.histplot(p_android_packageinstaller_0)

In [ ]:
sns.histplot(p_android_packageinstaller_1)

Vidíme že aj po vyhodení outlierov sa nám distribúcia nezmenila a preto môžeme pokračovať s testovaním.

In [ ]:
_ = sm.ProbPlot(p_android_packageinstaller_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(p_android_packageinstaller_1, fit=True).qqplot(line='45')

Ani `QQ-plot` nám nepotvrdil normálnu distribúciu. Pretože aby bola distribúcia normálna, tak by sa body museli nachádzať na priamke definovanej ako $x=y$.

In [ ]:
p_android_packageinstaller_0.count()

In [ ]:
p_android_packageinstaller_1.count()

Musíme použiť štatistický test, ktorý nám povie či dáta majú normálnu distribúciu alebo nie. Použijeme `Shapiro-Wilk` test pre jednu premennú a `Kolmogorov-Smirnov` pre druhú premennú pretože pri jednej je počet záznamov je nižší ako 5000 a pri druhej vyšší.

In [ ]:
# Shapiro-Wilk test
print_normality_test(p_android_packageinstaller_0)

In [ ]:
# Kolmogorov-Smirnov test
print_normality_test(p_android_packageinstaller_1)

Výsledok testu nám potvrdil, že dáta nemajú normálnu distribúciu, preto musíme použiť neparametrický test na overenie hypotézy. Pretože p hodnota je menšia ako 0.05.

Použijeme `Mann-Whitney U Test` pretože máme práve dve premenné a zároveň spĺňajú predpoklady pre tento test (vzorka musí byť aspoň 20).

In [ ]:
stats.mannwhitneyu(p_android_packageinstaller_0, p_android_packageinstaller_1)

Keďže nám vyšlo p menšie ako 0.05 tak zamietame nulovú hypotézu a prijímame alternatívnu hypotézu, že premenná `p.android.packageinstaller` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave.

#### 1.3.B Overenie štatistickej sily

Kedže nám p values vyšli všetky menšie ako 0.001, tak môžeme povedať že naše testy mali veľkú štatistickú silu.

## Zdroje

Prednášky a cvičenia z predmetu IAU.

[IAU Github repozitár](https://github.com/FIIT-IAU/IAU-course)

[Scipy dokumentácia](https://docs.scipy.org/doc/scipy/reference/index.html)

[Numpy dokumentácia](https://numpy.org/doc/)

[Pandas dokumentácia](https://pandas.pydata.org/docs/)

https://work.thaslwanter.at/Stats/html/statsAnalysis.html
